In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning import Trainer
import frnn
import wandb
import math
sys.path.append('../..')

from LightningModules.EdgeEmbedding.utils import graph_intersection, build_neighbors_list

device = "cuda" if torch.cuda.is_available() else "cpu"
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
input_dir = "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_filter/filter_processed/test"

In [4]:
event = torch.load(input_dir + "/10003")

In [5]:
def get_eff_and_pur(event, scorecut):
    positive_neighbors = event.idxs_scores > scorecut
    ind = torch.arange(len(event.pid), device = device).unsqueeze(1).expand(event.idxs.shape)
    found_edges = torch.stack([ind[positive_neighbors], event.idxs[positive_neighbors]], dim = 0)
    e_bidir = torch.cat([event.modulewise_true_edges, event.modulewise_true_edges.flip(0)], dim = 1)
    e_bidir_nocut = e_bidir.clone()
    e_bidir = e_bidir[:, (event.pt[e_bidir] > 1000.).any(0)]
    new_found_edges_nocut, y_nocut = graph_intersection(found_edges, e_bidir_nocut)
    true_positive_edges_nocut = new_found_edges_nocut[:,y_nocut == 1]
    new_found_edges, y = graph_intersection(found_edges, e_bidir)
    true_positive_edges = new_found_edges[:,y == 1]
    neighbor_list = build_neighbors_list(event.pid, true_positive_edges)
    neighbor_list_nocut = build_neighbors_list(event.pid, true_positive_edges_nocut)
    truth_neighbor_list = build_neighbors_list(event.pid, e_bidir)
    neighbor_count = (neighbor_list >= 0).sum(-1)
    neighbor_count_nocut = (neighbor_list_nocut >= 0).sum(-1)
    truth_neighbor_count = (truth_neighbor_list >= 0).sum(-1)
    all_neighbor_count = positive_neighbors.sum(-1)
    eff = ((neighbor_count - 1)*neighbor_count).sum()/((truth_neighbor_count-1)*truth_neighbor_count).sum()
    pur = ((neighbor_count - 1)*neighbor_count).sum()/((all_neighbor_count-1)*all_neighbor_count).sum()
    return eff.item(), pur.item(), ((y==1).sum()/e_bidir.shape[1]).item(), ((y_nocut==1).sum()/len(y)).item()

In [7]:
get_eff_and_pur(event, 0.296)

(0.9069286584854126,
 9.379954281030223e-05,
 0.9479787945747375,
 0.019468411803245544)

In [1]:
import cupy

<module 'cupy' from '/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/cupy/__init__.py'>